In [105]:
import numpy as np
import h5py
import os

from datasets import load_dataset
import polars as pl

In [106]:
print(np.__version__)
print(pl.__version__)

1.24.2
1.18.0


## Get list of files

In [82]:
raw_dir = "/home/vikas/Desktop/Globus/gaia"

In [83]:
def get_files(raw_dir, ext):

    files_found = []

    for path, dirs, files in os.walk(raw_dir):
        for file in files:
            if file.endswith(ext):
                files_found.append(os.path.join(path, file))

    return files_found

In [80]:
# Define the sorting key function
def healpix_sort_key(path):
    
    # Extract the number after 'healpix=' using string manipulation
    healpix_part = path.split('healpix=')[1]
    healpix_number = int(healpix_part.split('/')[0])
    
    return healpix_number

In [81]:
def sorted_files(start, end):
    
    files_hdf = get_files(raw_dir, ".hdf5")
    
    files_hdf_sorted = sorted(files_hdf, key = healpix_sort_key)
    
    return files_hdf_sorted[start:end]    

## Load dataset

In [128]:
def load_data(start, end):
    
    # Total number of files/shards is 1882    
    data_files = {
    "train": sorted_files(start, end)
    }
    
    hdf_dset = load_dataset(raw_dir,
                            split = 'train',
                            num_proc = 12,
                            streaming = False,
                            data_files = data_files)
    
    #hdf_dset = hdf_dset.with_format('numpy')
    
    return hdf_dset   

## Convert to Polars DataFrame

In [129]:
def get_object_id(hdf_dset):
    
    object_id_all = hdf_dset[:]["object_id"]
    
    object_series = pl.Series("object_id", object_id_all)
    
    return object_series    

In [135]:
def dset_to_polars(hdf_dset, feature):
    
    feature_all = hdf_dset[:][feature]
    
    df_feature_all = pl.DataFrame(feature_all)
    
    df_feature_all.insert_column(0, object_series)
    
    return df_feature_all   

In [136]:
hdf_dset = load_data(0, 10)

object_series = get_object_id(hdf_dset)

df_photometry = dset_to_polars(hdf_dset, "photometry") 

## Convert to parquet

In [25]:
dest_dir = "/home/vikas/Desktop/Globus_parquet/Gaia"

In [ ]:
def hdf_to_parquet(raw_dir, dest_dir):

    files_hdf = get_files(raw_dir, ".hdf5")

    for file in files_hdf:
        df_in = pd.read_hdf(file)

        fname = file.split("/")[-2]
        dir_out = os.path.join(dest_dir, fname)

        if not os.path.isdir(dir_out):
            os.mkdir(dir_out)

        df_in.to_parquet


        
        